In [9]:
import numpy as np
import requests
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image
from PIL import Image
import sys
sys.path.append("/homes/syli/python/semi_lnm")
from model.model import MultiTaskResNet
import json
import torch

In [10]:
model = MultiTaskResNet()
checkpoint = torch.load("../saved/model/1026_2524/checkpoint-epoch88.pth")
model.load_state_dict(checkpoint['state_dict'])
print(model)

MultiTaskResNet(
  (resnet): Sequential(
    (0): Conv3d(3, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=

In [7]:
# load data
from monai.data import CacheDataset
from monai.transforms import (
    Compose,
    EnsureChannelFirstd,
    ScaleIntensityd,
    CenterSpatialCropd,
    LoadImaged,
    CropForeground,
)
from Transform import Concat
val_transform = Compose([LoadImaged(["DWI", "T1CE", "T2", "DWI_roi", "T1CE_roi", "T2_roi"]),
                    EnsureChannelFirstd(["DWI", "T1CE", "T2", "DWI_roi", "T1CE_roi", "T2_roi"]), 
                    ScaleIntensityd(["DWI", "T1CE", "T2"]), 
                    CenterSpatialCropd(["DWI", "T1CE", "T2", "DWI_roi", "T1CE_roi", "T2_roi"], (200, 200, 12)), 
                    Concat(["DWI", "T1CE", "T2"]),
 ])
with open("/homes/syli/dataset/LVSI_LNM/dataset.json") as f:
        data = json.load(f)
val_ds = CacheDataset(data["validation"][:2], transform=val_transform, num_workers=1, cache_rate=0)    
input_tensor = val_ds[0]["image"]
print(input_tensor.shape)

(3, 200, 200, 12)


In [24]:
model.eval()
with torch.no_grad():
    print(model(torch.tensor(input_tensor[np.newaxis, ...])))

(tensor([[0.0053]]), tensor([[0.0021]]))


In [31]:
targets = [ClassifierOutputTarget(0)]
target_layers = [model.resnet[8]]
with GradCAM(model=model, target_layers=target_layers) as cam:
    grayscale_cams = cam(input_tensor=torch.tensor(input_tensor[np.newaxis, ...]), targets=targets)
    #cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)
    print(grayscale_cams.shape)
#images = np.hstack((np.uint8(255*img), cam , cam_image))


(1, 200, 12)


In [27]:
print(grayscale_cams.shape)

NameError: name 'grayscale_cams' is not defined